<a href="https://colab.research.google.com/github/Anidafio/4105_hw6/blob/main/hw6_p1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/Anidafio/4105_hw6/main/Housing.csv'

housing = pd.read_csv(url)

In [2]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

housing[varlist] = housing[varlist].apply(binary_map)

variables =  ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'parking']

X = housing[variables].copy().values
Y = housing[['price']].copy().values

from sklearn.preprocessing import MinMaxScaler
nc_X = MinMaxScaler()
X = nc_X.fit_transform(X)
Y = nc_X.fit_transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)

In [3]:
import torch

X_train = torch.tensor(X_train)
Y_train = torch.tensor(Y_train)
X_test = torch.tensor(X_test)
Y_test = torch.tensor(Y_test)

In [4]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [5]:
import torch.nn as nn
import torch.optim as optim

seq_model = nn.Sequential(
            nn.Linear(11, 32, dtype=torch.float64),
            nn.Tanh(),
            nn.Linear(32, 1, dtype=torch.float64))
seq_model

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 10000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = X_train,
    t_u_val = X_test,
    t_c_train = Y_train,
    t_c_val = Y_test)

#print('output', seq_model(X_test))
#print('answer', Y_test)
#print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 0.2249, Validation loss 0.2062
Epoch 1000, Training loss 0.0199, Validation loss 0.0191
Epoch 2000, Training loss 0.0159, Validation loss 0.0149
Epoch 3000, Training loss 0.0140, Validation loss 0.0130
Epoch 4000, Training loss 0.0128, Validation loss 0.0118
Epoch 5000, Training loss 0.0120, Validation loss 0.0110
Epoch 6000, Training loss 0.0114, Validation loss 0.0105
Epoch 7000, Training loss 0.0110, Validation loss 0.0101
Epoch 8000, Training loss 0.0107, Validation loss 0.0098
Epoch 9000, Training loss 0.0104, Validation loss 0.0095
Epoch 10000, Training loss 0.0102, Validation loss 0.0094
